In [ ]:
import ee
import geemap
import ipywidgets as widgets
from IPython.display import display, clear_output
from ipyleaflet import Map, TileLayer, CircleMarker, LayersControl, WidgetControl
from functools import partial
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import json
import uuid  # Use Python's built-in UUID library
import geemap.core as geemap_core

# --- 1. Initialize GEE ---
# --- 1. Initialize GEE ---

try:
    # Try to initialize without auth
    ee.Initialize(project='saltmarshes')
except Exception as e:
    # If it fails, create an auth button
    auth_button = geemap_core.auth_button()

    # Display the button. The app will PAUSE here until it's clicked.
    display(auth_button)

    # After the button is clicked, initialize
    ee.Initialize(project='saltmarshes')

# --- 2. Create the Initial UI for Drawing an ROI ---
print("Step 1: Use the drawing tool (▰) on the map below to draw your Region of Interest (ROI).")
print("Step 2: Click the 'Load Imagery for ROI' button.")

m_roi = geemap.Map(center=[0, 0], zoom=3, height="350px")
m_roi.draw_control.rectangle = {}
display(m_roi)

load_button = widgets.Button(description="Load Imagery for ROI", button_style="primary")
main_app_output = widgets.Output()

# --- 3. Define the Main App Logic (runs on button click) ---
def on_load_button_click(b):
    with main_app_output:
        main_app_output.clear_output()
        roi = m_roi.user_roi
        if roi is None:
            print("Error: No ROI drawn. Please draw a rectangle and try again.")
            return
        print("Loading imagery for your ROI... please wait.")

        # --- A. Get GEE Data (using geemap) ---
        alpha_earth = ee.ImageCollection("GOOGLE/SATELLITE_EMBEDDING/V1/ANNUAL").filterDate("2024-01-01", "2024-12-31").filterBounds(roi).mosaic().clip(roi)
        alpha_vis = {"bands": ["A01", "A16", "A09"], "min": -0.5, "max": 0.5}

        s2_img = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED").filterDate("2024-01-01", "2024-12-31").filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 20)).filterBounds(roi).median().clip(roi)
        s2_rgb_vis = {"bands": ["B4", "B3", "B2"], "min": 0, "max": 3000}
        s2_fc_vis = {"bands": ["B8", "B4", "B3"], "min": 0, "max": 3000}

        # --- B. Create a NATIVE ipyleaflet Map (This is the tessera method) ---
        center_coords = roi.centroid(1).coordinates().get(1).getInfo(), roi.centroid(1).coordinates().get(0).getInfo()
        m_main = Map(center=center_coords, zoom=12, height="500px")
        
        # --- C. Get Tile Layers from GEE ---
        s2_rgb_tiles = geemap.ee_tile_layer(s2_img, s2_rgb_vis, "Sentinel-2 RGB")
        s2_fc_tiles = geemap.ee_tile_layer(s2_img, s2_fc_vis, "Sentinel-2 False-Color")
        alpha_earth_tiles = geemap.ee_tile_layer(alpha_earth, alpha_vis, "AlphaEarth")
        google_satellite = TileLayer(
            url='https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
            name='Google Satellite',
            attribution='Google'
        )
        
        # --- D. Add Layers to the ipyleaflet Map ---
        m_main.add_layer(google_satellite)
        m_main.add_layer(s2_rgb_tiles)
        m_main.add_layer(s2_fc_tiles)
        m_main.add_layer(alpha_earth_tiles)
        m_main.add_control(LayersControl(position='topright'))

        # --- E. State Management (Client-Side) ---
        points_list = [] # List of point *dictionaries*
        markers_dict = {} # Dict of {uuid: CircleMarker object}
        
        # --- F. Create Point Collection UI ---
        class_list = [
            'saltmarsh', 'freshwater marsh', 'mangrove', 'forest', 
            'farmland', 'urban', 'seagrass', 'water', 'other'
        ]
        class_dropdown = widgets.Dropdown(options=class_list, value='saltmarsh', description='Class:')
        
        note_box = widgets.Text(placeholder='Optional note...', description='Note:')
        point_count_label = widgets.Label('Markers added: 0')
        
        # --- ### THIS IS THE FIRST FIX ### ---
        # We change 'VBox' to an 'Output' widget. It's designed for this and handles scrolling.
        preview_panel = widgets.Output(layout=widgets.Layout(
            height='150px', 
            overflow='scroll', # Use 'scroll' to force the scrollbar
            border='1px solid #ccc',
            padding='5px' # Add a little padding
        ))
        
        export_button = widgets.Button(description="Export Points to CSV", button_style="primary")
        export_output = widgets.Output()

        # --- G. Define Helper Functions (Core Logic from Tessera) ---
        
        # Helper to get colors (just like tessera)
        tab10_cmap = plt.colormaps.get_cmap("tab10")
        class_color_map = {}
        def get_or_assign_color(class_name):
            if class_name not in class_color_map:
                new_color_index = len(class_color_map) % 10
                class_color_map[class_name] = mcolors.to_hex(tab10_cmap(new_color_index))
            return class_color_map[class_name]

        # --- ### THIS IS THE SECOND FIX ### ---
        # We update this function to work with the new 'Output' widget
        def update_preview_panel():
            """Redraws the preview panel from the points_list."""
            rows = []
            for p in points_list:
                label = widgets.Label(f"{p['class']} ({p['latitude']:.3f}, {p['longitude']:.3f})")
                delete_btn = widgets.Button(description='Delete', button_style='danger', layout=widgets.Layout(width='80px'))
                delete_btn.on_click(lambda b, uuid_to_delete=p['uuid']: handle_marker_delete(uuid_to_delete)) 
                rows.append(widgets.HBox([label, delete_btn]))
            
            # This is the new, robust way to update the panel
            with preview_panel:
                preview_panel.clear_output(wait=True) # Clear previous content
                if rows:
                    display(widgets.VBox(rows)) # Display the new list of rows
                else:
                    display(widgets.Label("No points added yet."))
                    
            point_count_label.value = f'Markers added: {len(points_list)}'

        def add_training_point(lat, lon, class_name):
            """Adds a new point to the list and a new marker to the map."""
            uuid_str = str(uuid.uuid4())
            new_point = {
                'class': class_name, 'note': note_box.value,
                'latitude': lat, 'longitude': lon, 'uuid': uuid_str
            }
            points_list.append(new_point)
            
            color = get_or_assign_color(class_name)
            marker = CircleMarker(
                location=(lat, lon), radius=5, color=color,
                fill_color=color, fill_opacity=0.6, name=class_name
            )
            marker.on_click(lambda **kwargs: handle_marker_delete(uuid_str))
            markers_dict[uuid_str] = marker
            m_main.add_layer(marker)
            update_preview_panel()
            note_box.value = '' # Clear note box

        def handle_map_interaction(**kwargs):
            """This is the function that runs on any map interaction."""
            if kwargs.get('type') == 'click':
                coords = kwargs.get('coordinates') # This gives (lat, lon)
                add_training_point(coords[0], coords[1], class_dropdown.value)

        def handle_marker_delete(uuid_to_delete):
            """Finds a point by its UUID and removes it."""
            nonlocal points_list
            if uuid_to_delete in markers_dict:
                m_main.remove_layer(markers_dict[uuid_to_delete])
                del markers_dict[uuid_to_delete]
            points_list = [p for p in points_list if p['uuid'] != uuid_to_delete]
            update_preview_panel()

        def export_points(b):
            """Exports the points_list to a CSV."""
            with export_output:
                export_output.clear_output()
                if not points_list:
                    print("No points to export.")
                    return
                print("Exporting... please wait.")
                
                ee_features = [ee.Feature(ee.Geometry.Point(p['longitude'], p['latitude']), p) for p in points_list]
                fc = ee.FeatureCollection(ee_features)
                
                filename = "collected_points.csv"
                try:
                    # Use the stable Google Drive export function
                    geemap.ee_export_vector(
                        fc, 
                        filename=filename, 
                        selectors=["class", "note", "latitude", "longitude"]
                    )
                    print(f"Success! Check your Google Drive for '{filename}'. It may take a moment to appear.")
                except Exception as e:
                    print(f"Export Failed: {e}")

        # --- H. Connect Handlers and Display ---
        m_main.on_interaction(handle_map_interaction) 
        export_button.on_click(export_points)
        update_preview_panel() # Call it once to show "No points added yet."
        
        print("App is ready! Use the layer menu (top-right) to switch imagery.")
        display(m_main)
        display(widgets.VBox([
            widgets.Label("Step 3: Label Points"),
            widgets.Label("Select a class, then click on the map to add a marker."),
            class_dropdown,
            note_box,
            point_count_label,
            preview_panel,
            export_button,
            export_output
        ]))

# --- 4. Link Button and Display Initial UI ---
load_button.on_click(on_load_button_click)
display(load_button, main_app_output)

Step 1: Use the drawing tool (▰) on the map below to draw your Region of Interest (ROI).
Step 2: Click the 'Load Imagery for ROI' button.


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', transp…

Button(button_style='primary', description='Load Imagery for ROI', style=ButtonStyle())

Output()